문서분류_스팸 필터링 예제

In [131]:
training_set = [ ["me free lottery", 1], 
["free get free you", 1],
["you free scholarship", 0],
["free to contact me", 0],
["you won award", 0],
["you ticket lottery", 1]]

In [132]:
#spam:1, normal:0
#dict={'award' : [1,0], ...} ~~각 단어에 대해 normal인것과 spam인것 분류
#cnt_0
#cnt_1

from collections import defaultdict
normal_cnt = 0
spam_cnt = 0

wordfreq = defaultdict(lambda : [0,0]) #key가 없으면 값을 [0,0]으로 세팅
for doc, label in training_set:
    words = doc.split()
    for word in words:
        wordfreq[word][label] +=1 #스팸) label이 1이면 1번째에 +1
print(wordfreq)

for key, (cnt0, cnt1) in wordfreq.items():
    normal_cnt += cnt0
    spam_cnt += cnt1
print(normal_cnt)
print(spam_cnt)

defaultdict(<function <lambda> at 0x00000172AC970B88>, {'me': [1, 1], 'free': [2, 3], 'lottery': [0, 2], 'get': [0, 1], 'you': [2, 2], 'scholarship': [1, 0], 'to': [1, 0], 'contact': [1, 0], 'won': [1, 0], 'award': [1, 0], 'ticket': [0, 1]})
10
10


Laplace Smoothing

In [133]:
#wordprob = {"award" : [0.1364,0.0455], ...}
k=0.5
wordprob = defaultdict(lambda : [0,0])

for key, (cnt0,cnt1) in wordfreq.items():
    wordprob[key][0] = (k + cnt0) / (2*k + normal_cnt)
    wordprob[key][1] = (k + cnt1) / (2*k + spam_cnt)
wordprob

defaultdict(<function __main__.<lambda>()>,
            {'me': [0.13636363636363635, 0.13636363636363635],
             'free': [0.22727272727272727, 0.3181818181818182],
             'lottery': [0.045454545454545456, 0.22727272727272727],
             'get': [0.045454545454545456, 0.13636363636363635],
             'you': [0.22727272727272727, 0.22727272727272727],
             'scholarship': [0.13636363636363635, 0.045454545454545456],
             'to': [0.13636363636363635, 0.045454545454545456],
             'contact': [0.13636363636363635, 0.045454545454545456],
             'won': [0.13636363636363635, 0.045454545454545456],
             'award': [0.13636363636363635, 0.045454545454545456],
             'ticket': [0.045454545454545456, 0.13636363636363635]})

log 이용한 underflow 방지

In [134]:
#교안 p.42 수식 참고
doc = "free lottery"
tokens = doc.split()

import math
log_prob0, log_prob1 = 0.0, 0.0 #초기값

for word, (prob0,prob1) in wordprob.items():
    if word in tokens:
        log_prob0 += math.log(prob0)
        log_prob1 += math.log(prob1)
        
log_prob0 += math.log(normal_cnt / (normal_cnt + spam_cnt))
log_prob1 += math.log(spam_cnt / (normal_cnt + spam_cnt))

prob0 = math.exp(log_prob0)
prob1 = math.exp(log_prob1)

print(prob0)
print(prob1)

0.00516528925619835
0.03615702479338842


In [135]:
print("정상확률{:.1f}%".format(prob0 / (prob0 + prob1) * 100))
print("스팸확률{:.1f}%".format(prob1 / (prob0 + prob1) * 100))

정상확률12.5%
스팸확률87.5%


들어오는 단어에 따라 정상/스팸 확률 다르게 구하는 함수

In [139]:
def get_prob(doc):
    tokens = doc.split()

    log_prob0, log_prob1 = 0.0, 0.0

    for word, (prob0,prob1) in wordprob.items():
        if word in tokens:
            log_prob0 += math.log(prob0)
            log_prob1 += math.log(prob1)

    log_prob0 += math.log(normal_cnt / (normal_cnt + spam_cnt))
    log_prob1 += math.log(spam_cnt / (normal_cnt + spam_cnt))

    prob0 = math.exp(log_prob0)
    prob1 = math.exp(log_prob1)
    
    return prob0,prob1

doc = 'me ticket'
prob0, prob1 = get_prob(doc)

print(prob0)
print(prob1)

0.0030991735537190075
0.009297520661157025
